In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data, 'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
column_names = ['PostalCode', 'Borough', 'Neighborhood']
rows_list = []
for tr in My_table.findAll('tr'):
    tds = tr.findAll('td')
    if not tds:
        continue
    postcode, borough, neighborhood = [td.text.strip() for td in tds[:3]]
    rows_list.append([postcode, borough, neighborhood])
postalcodes = pd.DataFrame(rows_list, columns=column_names)
indexNames = postalcodes[postalcodes['Borough'] == 'Not assigned'].index
postalcodes.drop(indexNames, inplace=True)
postalcodes = postalcodes.reset_index()
del postalcodes['index']
postalcodes.loc[postalcodes['Neighborhood'] == ('Not assigned'), 'Neighborhood'] = postalcodes['Borough']
postalcodes_group = postalcodes.groupby(['PostalCode', 'Borough'], sort=False).agg(lambda x: ', '.join(x))
postalcodes_group = postalcodes_group.reset_index()

In [8]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
merged_dataframe = pd.merge(postalcodes_group, coordinates_df, how='left', on='PostalCode')
merged_dataframe_Toronto = merged_dataframe[merged_dataframe['Borough'].str.contains("Toronto")]
merged_dataframe_Toronto = merged_dataframe_Toronto.reset_index()
del merged_dataframe_Toronto['index']

merged_dataframe_Toronto.shape
dfToronto = merged_dataframe_Toronto
dfToronto.head()
#dfToronto.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [13]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
#geolocator 
location = geolocator.geocode(address)
location
# latitude = location.latitude
# longitude = location.longitude
# print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

GeocoderTimedOut: Service timed out

In [11]:
for lat, lng, postalcode, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['PostalCode'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}, {}'.format(postalcode, borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

NameError: name 'map_toronto' is not defined